# MODULES

In [115]:
import numpy as np
import pandas as pd
import math

import plotly.graph_objects as go

from scipy.spatial import distance
from scipy.stats import cauchy
import scipy.stats as stats

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

# CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# Activity 1: Path length - (BM1 vs BM2 vs CRW)
- Write a function that returns a Brownian Motion (BM) trajectory in pandas df.
- Write a function that returns a Correlated Random Walk (CRW) trajectory in pandas df.
- Write a function that returns the path length for a given trajectory.
- Compare at least the path length of three trajectories as shown in the figure below.
- Display the results using plotly.

In [3]:
met_pl = pd.read_csv('met_df_1.csv')
met_pl

,pl_BM_3,pl_BM_6,pl_CRW_6
0,3.0,6.0,6.0
1,6.0,12.0,12.0
2,9.0,18.0,18.0
3,12.0,24.0,24.0
4,15.0,30.0,30.0
...,...,...,...
994,2985.0,5970.0,5970.0
995,2988.0,5976.0,5976.0
996,2991.0,5982.0,5982.0
997,2994.0,5988.0,5988.0


In [26]:

# Init figure
fig_met_1 = go.Figure()

fig_met_1.add_trace(go.Scatter(
    x = met_pl.index,
    y = met_pl.pl_BM_3,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'Path length_BM_3',
    showlegend = True))

fig_met_1.add_trace(go.Scatter(
    x = met_pl.index,
    y = met_pl.pl_BM_6,
    marker = dict(size=2),
    line = dict(width=4),
    mode = 'lines',
    name = 'Path length_BM_6',
    showlegend = True))

fig_met_1.add_trace(go.Scatter(
    x = met_pl.index,
    y = met_pl.pl_CRW_6,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'Path length_CRW_6',
    showlegend = True))

fig_met_1.update_layout(title='Path length - (BM1 vs BM2 vs CRW)')

fig_met_1.show()

# Mean Squared Displacement

# Activity 2: Mean Squared Displacement - (BM vs CRW)
- Write a function that returns the mean squared displacement for a given trajectory.
- Compare the mean squared displacement curves of at least two trajectories of different kinds, as shown in the figure below.
- Display the results using plotly.

In [5]:
met_msd = pd.read_csv('met_df_2.csv')

In [6]:
fig_met_msd = go.Figure()

fig_met_msd.add_trace(go.Scatter(
    x = met_msd.index,
    y = met_msd.MSD_BM,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'MSD_BM',
    showlegend = True))

fig_met_msd.add_trace(go.Scatter(
    x = met_msd.index,
    y = met_msd.MSD_CRW,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'MSD_CRW',
    showlegend = True))

fig_met_msd.show()

In [25]:
def calculate_msd(trajectory):
    N = len(trajectory)
    msd = np.zeros(N)

    # Calculate MSD
    for dt in range(1, N):
        displacements = trajectory[dt:] - trajectory[:-dt]
        squared_displacements = np.sum(displacements**2, axis=1)
        msd[dt] = np.mean(squared_displacements)
    
    return msd

def random_walk(N, dim=2):
    steps = np.random.randn(N, dim)
    return np.cumsum(steps, axis=0)

def linear_trajectory(N, velocity, dim=2):
    t = np.arange(N)
    return np.array([velocity * t, velocity * t]).T

n_steps= 1000
# create diferents trajectories
random_walk_traj = random_walk(n_steps)
linear_traj = linear_trajectory(n_steps, velocity=0.1, dim=2)

# Calculate MSD for each trajectory
msd_random_walk = calculate_msd(random_walk_traj)
msd_linear = calculate_msd(linear_traj)

#plot with plotly
figure = go.Figure()

figure.add_trace(go.Scatter(
    x= np.arange(n_steps), 
    y= msd_random_walk, 
    mode='lines', 
    name='Random Walk',
    showlegend = True))

figure.add_trace(go.Scatter(
    x= np.arange(n_steps), 
    y= msd_linear, 
    mode='lines', 
    name='Lineal trajectory',
    showlegend = True))

figure.update_layout(title='Mean Squared Displacement')

figure.show()


# Turning-angle distribution

# Activity 3: Turning-angle Distribution - (source dist. vs observed dist.)
- Consider two CRW trajectories with different Cauchy coefficients.
- Write a function that returns the turning angles for a given trajectory.
- Compare the observed distribution (histogram) to the source distribution (curve) for both trajectories, as shown in the figure below.
- Display the results using plotly.

In [19]:
def crw_trajectory(N, scale, dim=2):
    angles = cauchy.rvs(scale=scale, size=N)  #Cauchy distribution Angles
    trajectory = np.zeros((N, dim))
    
    # Init random walk
    direction = np.random.rand() * 2 * np.pi
    for i in range(1, N):
        direction += angles[i]  #change angle direction
        trajectory[i] = trajectory[i-1] + np.array([np.cos(direction), np.sin(direction)])
    
    return trajectory, angles
#Calculate turning angle 
def turning_angles(trajectory):
    angles = []
    for i in range(1, len(trajectory)-1):
        vec1 = trajectory[i] - trajectory[i-1]
        vec2 = trajectory[i+1] - trajectory[i]
        
        angle = np.arctan2(vec2[1], vec2[0]) - np.arctan2(vec1[1], vec1[0])
        angles.append(angle)
        
    return np.array(angles)
#plot the histograms
def turning_angles_histogram(angles, cauchy_scale, name):
    
    hist = np.histogram(angles, bins=200, density=True)
    
    x = np.linspace(-np.pi, np.pi, 100)
    pdf = cauchy.pdf(x, scale=cauchy_scale)
    
    # create it using Plotly
    trace_hist = go.Bar(x=hist[1][:-1], y=hist[0], name=f'Observed ({name})', opacity=0.8)
    trace_pdf = go.Scatter(x=x, y=pdf, mode='lines', name=f'Cauchy ({name})')
    
    return [trace_hist, trace_pdf]

# create CRW trajectories wit diferents cauchy coeficients
N = 2000
scale_1 = 0.6
scale_2 = 0.9

traj_1, angles_1 = crw_trajectory(N, scale=scale_1)
traj_2, angles_2 = crw_trajectory(N, scale=scale_2)

#Calculate turning-angle for both trajectories.
turning_angles_1 = turning_angles(traj_1)
turning_angles_2 = turning_angles(traj_2)

fig = go.Figure()

#Add the histogram for 0.6 cauchy coeficient
fig.add_traces(turning_angles_histogram(turning_angles_1, scale_1, name='0.6'))

#Add the histogram for 0.9 cauchy coeficient
fig.add_traces(turning_angles_histogram(turning_angles_2, scale_2, name='0.9'))

#layout
fig.update_layout(title='Turning-angle Distribution',
                  xaxis=dict(title='Angle'),
                  yaxis=dict(title='Density'))
                  
fig.show()


# Activity 4: Step-length Distribution - (source dist. vs observed dist.)
- Write a function that returns a Lévy Walk (LW) trajectory in pandas df.
- Consider two LW trajectories with different alpha coefficients.
- Write a function that restaurants the step lengths for a given trajectory.
- Compare the observed distribution (histogram) to the source distribution (curve) for both trajectories, as shown in the figure below.
- Display the results using plotly.

In [153]:
#Levy parameters
alpha_1 = 1.0
beta_1 = 1.0
alpha_2 = 0.8
beta_2 = 1.0
sample_size = 200

#Create random Levy parameters for #1 Using Pandas
data_1 = levy_stable.rvs(alpha_1, beta_1, size=sample_size)
df_1 = pd.DataFrame({'levy_data_1': data_1})

#Create random Levy parameters for #2 Using Pandas
data_2 = levy_stable.rvs(alpha_2, beta_2, size=sample_size)
df_2 = pd.DataFrame({'levy_data_2': data_2})

#Create histogrm
fig = go.Figure()

fig.add_trace(go.Histogram(x=df_1['levy_data_1'], nbinsx=1000, histnorm='probability density', 
                           name=f'Observed_alpha={alpha_1}, beta={beta_1}', opacity=0.6))

fig.add_trace(go.Histogram(x=df_2['levy_data_2'], nbinsx=1000, histnorm='probability density', 
                           name=f'Observed_alpha={alpha_2}, beta={beta_2}', opacity=0.6))

#Obtain first levi distribution
x_values_1 = np.linspace(df_1['levy_data_1'].min(), df_1['levy_data_1'].max(), 1000)
pdf_1 = levy_stable.pdf(x_values_1, alpha_1, beta_1)

#Obtain second levi distribution
x_values_2 = np.linspace(df_2['levy_data_2'].min(), df_2['levy_data_2'].max(),1000)
pdf_2 = levy_stable.pdf(x_values_2, alpha_2, beta_2)

#Add curve #1
fig.add_trace(go.Scatter(
    x=x_values_1, 
    y=pdf_1, 
    mode='lines', 
    name=f'Levy_alpha={alpha_1}, beta={beta_1}', 
    line=dict(color='blue')))

# Add curve #2
fig.add_trace(go.Scatter(
    x=x_values_2,
    y=pdf_2, 
    mode='lines', 
    name=f'Levy_alpha={alpha_2}, beta={beta_2}',
    line=dict(color='red')))

#layers
fig.update_layout(
    title='Step-length Distribution',
    showlegend=True,)

fig.update_traces(opacity=0.5)

fig.show()
